In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
pd.options.display.max_columns = None

In [2]:
os.chdir(r"C:\Users\RKrishna\OneDrive - National Instruments\Data\Project\Email Spam Classification")
os.getcwd()

'C:\\Users\\RKrishna\\OneDrive - National Instruments\\Data\\Project\\Email Spam Classification'

In [3]:
#importing email dataset
df = pd.read_csv("emails.csv")
df.shape

(5728, 2)

In [4]:
#Removing Duplicates
df = df.drop_duplicates()
df.shape

(5695, 2)

In [5]:
#Checking for missing values
df.isnull().sum() #no missing values

text    0
spam    0
dtype: int64

In [6]:
df['text'] = df['text'].map(lambda text: text[9:])
df['text'] = df['text'].map(lambda text:re.sub('[^a-zA-Z0-9]+', ' ',text)).apply(lambda x: (x.lower()).split())
df

,text,spam
0,"[naturally, irresistible, your, corporate, ide...",1
1,"[the, stock, trading, gunslinger, fanny, is, m...",1
2,"[unbelievable, new, homes, made, easy, im, wan...",1
3,"[4, color, printing, special, request, additio...",1
4,"[do, not, have, money, get, software, cds, fro...",1
...,...,...
5723,"[re, research, and, development, charges, to, ...",0
5724,"[re, receipts, from, visit, jim, thanks, again...",0
5725,"[re, enron, case, study, update, wow, all, on,...",0
5726,"[re, interest, david, please, call, shirley, c...",0


In [7]:
ps = PorterStemmer()
stops = stopwords.words("English")

In [8]:
corpus=df['text'].apply(lambda text_list:' '.join(list(map(lambda word:ps.stem(word),(list(filter(lambda text:text not in set(stopwords.words('english')),text_list)))))))

In [9]:
df

,text,spam
0,"[naturally, irresistible, your, corporate, ide...",1
1,"[the, stock, trading, gunslinger, fanny, is, m...",1
2,"[unbelievable, new, homes, made, easy, im, wan...",1
3,"[4, color, printing, special, request, additio...",1
4,"[do, not, have, money, get, software, cds, fro...",1
...,...,...
5723,"[re, research, and, development, charges, to, ...",0
5724,"[re, receipts, from, visit, jim, thanks, again...",0
5725,"[re, enron, case, study, update, wow, all, on,...",0
5726,"[re, interest, david, please, call, shirley, c...",0


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus.values).toarray()
y = df.iloc[:, 1].values

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [12]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
classifier.fit(X_train , y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
y_pred = classifier.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_test, y_pred)
CM

array([[861,   9],
       [  1, 268]], dtype=int64)

In [15]:
#Confusion Matrix
CM = pd.crosstab(y_test, y_pred)
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]
Accuracy_NB=((TP+TN)*100)/(TP+TN+FP+FN)
#True negative rate
TNR_NB=TN*100/(TN+FP)
#Recall:
#Recall is positive cases correctly identified
Recall_NB=TP*100/(TP+FN)
#False Positive rate:
#Number of false positives predicted:
FPR_NB=FP*100/(FP+TN)
#False Negative rate:
#Number of false negatives predicted
FNR_NB=FN*100/(FN+TP)
#Accuracy_DT
print("Decision Tree")
print("Accuracy: "+ str(Accuracy_NB))
print("TNR: "+ str(TNR_NB))
print("TPR: "+ str(Recall_NB))
print("FPR: "+ str(FPR_NB))
print("FNR: "+ str(FNR_NB))

Decision Tree
Accuracy: 99.12203687445127
TNR: 98.96551724137932
TPR: 99.62825278810409
FPR: 1.0344827586206897
FNR: 0.37174721189591076
